In [0]:
import os
import keras
os.environ['CUDA_VISIBLE_DEVICES']='0'

import numpy as np
import cv2
import matplotlib.pyplot as plt

train,test=keras.datasets.cifar10.load_data()

In [0]:
x_train,y_train=train
x_test,y_test=test
y_train=y_train.astype(int)
y_test=y_test.astype(int)

In [0]:
x_train_histogram = []
x_test_histogram = []

for i in range(len(x_train)):
    chans = cv2.split(x_train[i]) 
    hist_feature = []
    for chan in chans:
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) 
        hist_feature.extend(hist.flatten())
    x_train_histogram.append(hist_feature)

for i in range(len(x_test)):
    chans = cv2.split(x_test[i])
    hist_feature = []
    for chan in chans:
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) 
        hist_feature.extend(hist.flatten())
    x_test_histogram.append(hist_feature)

x_train_histogram = np.array(x_train_histogram)
x_test_histogram = np.array(x_test_histogram)


In [0]:
bin_n=16

def hog(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist.astype(np.float32)

x_train_hog=np.array([hog(x) for x in x_train])
x_test_hog=np.array([hog(x) for x in x_test])

In [0]:
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(5.383)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(2.67)

SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, y_train)

_, y_hist_train = SVM_hist.predict(x_train_histogram)
_, y_hist_test = SVM_hist.predict(x_test_histogram)